In [ ]:
import os
import pickle
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Conv3D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split


# Dataset path
dataset_path = r"C:\Users\PMLS\Documents\2ndSemester\CV\cvProject\PSL\dynamic_data"

# Configuration
clip_length = 32
frame_size = (64, 64)
batch_size = 10
epochs = 10
model_save_path = "urdu_sign_convlstm_model.pkl"

# Label Mapping
english_to_urdu = {
    "2-Hay": "ہ",
    "Alifmad": "آ",
    "Aray": "ڑ",
    "Jeem": "ج"
}

# Preprocessing Videos
def preprocess_videos(dataset_path, clip_length, frame_size):
    clips = []
    labels = []
    for folder_name in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder_name)
        if os.path.isdir(folder_path):
            label = english_to_urdu.get(folder_name)
            if label is None:
                continue

            for video_name in os.listdir(folder_path):
                video_path = os.path.join(folder_path, video_name)
                cap = cv2.VideoCapture(video_path)
                frames = []
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    resized_frame = cv2.resize(frame, frame_size)
                    frames.append(resized_frame)
                cap.release()

                # Create clips of fixed length
                for i in range(0, len(frames) - clip_length + 1, clip_length):
                    clip = frames[i:i + clip_length]
                    clips.append(clip)
                    labels.append(label)
    clips = np.array(clips, dtype=np.float32) / 255.0  # Normalize pixel values
    return clips, labels

# Load and preprocess the dataset
print("Preprocessing videos...")
clips, labels = preprocess_videos(dataset_path, clip_length, frame_size)

# Convert Urdu labels to integers and one-hot encode
unique_labels = list(set(labels))
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
indexed_labels = [label_to_index[label] for label in labels]
encoded_labels = to_categorical(indexed_labels, num_classes=len(unique_labels))

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(clips, encoded_labels, test_size=0.2, random_state=42)

# Build the ConvLSTM Model
def build_convlstm(input_shape, num_classes):
    model = Sequential([
        ConvLSTM2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', return_sequences=True, input_shape=input_shape),
        ConvLSTM2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', return_sequences=False),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model Configuration
input_shape = (clip_length, frame_size[0], frame_size[1], 3)
num_classes = len(unique_labels)

print("Building ConvLSTM model...")
model = build_convlstm(input_shape, num_classes)

# Train the Model
print("Training model...")
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

# Save the Model
model_save_path = 'convlstm_model.pkl'
with open(model_save_path, 'wb') as f:
    pickle.dump(model, f)

print(f"Model saved at {model_save_path}")
